In [ ]:
import snowflake.connector
import pandas as pd
import matplotlib.pyplot as plt
import kaggle


In [ ]:
# Download the dataset
!kaggle datasets download -d hanaksoy/customer-purchasing-behaviors

# Unzip and load into Snowflake
import zipfile
with zipfile.ZipFile('customer-purchasing-behaviors.zip', 'r') as zip_ref:
    zip_ref.extractall('.')


In [ ]:
# Snowflake connection details
def create_snowflake_connection():
    try:
        conn = snowflake.connector.connect(
            user='YOUR_USER',
            password='YOUR_PASSWORD',
            account='YOUR_ACCOUNT',
            role='YOUR_ROLE',
            warehouse='YOUR_WAREHOUSE',
            database='YOUR_DATABASE',
            schema='YOUR_SCHEMA'
        )
        return conn
    except snowflake.connector.errors.ProgrammingError as e:
        print(f"Programming error: {e}")
    except snowflake.connector.errors.DatabaseError as e:
        print(f"Database error: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")


In [ ]:
# Create a table in Snowflake
create_table_query = """
CREATE TABLE customer_purchasing_behavior (
    customer_id INT,
    age INT,
    location STRING,
    purchase_amount FLOAT,
    purchase_date DATE
)
"""
conn.cursor().execute(create_table_query)


In [ ]:
# Fetch data from Snowflake
def fetch_stock_prices():
    conn = create_snowflake_connection()
    query = "SELECT * FROM stock_schema.stock_prices;"
    data = pd.read_sql(query, conn)
    conn.close()
    return data


In [ ]:
# Load and display data
data = fetch_stock_prices()
data.head()

In [ ]:
# Visualize OPEN_PRICE and CLOSE_PRICE
plt.figure(figsize=(10, 5))
plt.plot(data['trade_date'], data['open_price'], label='Open Price', marker='o')
plt.plot(data['trade_date'], data['close_price'], label='Close Price', marker='x')
plt.title('Open vs Close Prices')
plt.xlabel('Trade Date')
plt.ylabel('Price')
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Additional Visualizations
# Purchasing Trends
plt.figure(figsize=(10, 5))
plt.plot(data['purchase_date'], data['purchase_amount'], label='Purchase Amount', marker='o')
plt.title('Purchasing Trends')
plt.xlabel('Purchase Date')
plt.ylabel('Amount')
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

# Customer Segments
segment_data = data.groupby('age')['purchase_amount'].sum().reset_index()
plt.figure(figsize=(10, 5))
plt.bar(segment_data['age'], segment_data['purchase_amount'])
plt.title('Purchase Totals by Age Group')
plt.xlabel('Age Group')
plt.ylabel('Total Purchase Amount')
plt.tight_layout()
plt.show()

# Top Customers
top_customers = data.groupby('customer_id')['purchase_amount'].sum().nlargest(10).reset_index()
top_customers